<a href="https://colab.research.google.com/github/umjiwoo/ubion_hackathon/blob/main/modeling_jiwoo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

modeling 공통

모델을 코랩에서 돌리든 비쥬얼스튜디오로 돌리든 최종 모델은 파일로 저장해오기로 하자(.h5)

모든 모델의 random_state=1로 하기로 함

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ubion_hackathon/train_data.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ubion_hackathon/test_data.csv")

# train = pd.read_csv("./datasets/train_data.csv")
# test = pd.read_csv("./datasets/test_data.csv")

In [4]:
y_train = train["TARGET"]
X_train = train.drop("TARGET", axis=1)

In [5]:
y_test = test["TARGET"]
X_test = test.drop("TARGET", axis=1)

In [6]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(203604, 73) (203604,) (50939, 73) (50939,)


#### 오버샘플링

In [7]:
from imblearn.over_sampling import SMOTE
# 검증 데이터나 테스트 데이터가 아닌 학습데이터에서만 오버샘플링 사용할 것
smote = SMOTE(random_state=11)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [8]:
y_train_smote

0         0
1         0
2         0
3         0
4         0
         ..
374005    1
374006    1
374007    1
374008    1
374009    1
Name: TARGET, Length: 374010, dtype: int64

In [9]:
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_smote.shape, y_train_smote.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_smote).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (203604, 73) (203604,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (374010, 73) (374010,)
SMOTE 적용 후 레이블 값 분포: 
 0    187005
1    187005
Name: TARGET, dtype: int64


#### 오버&언더 샘플링

In [10]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks


smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_train_over, y_train_over = smoteto.fit_resample(X_train, y_train)

#### 모델 평가 함수
#### get_clf_eval

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred):
    test_cm = confusion_matrix(y_test, pred)
    test_acc = accuracy_score(y_test, pred)
    test_prc = precision_score(y_test, pred)
    test_rcll = recall_score(y_test, pred)
    test_f1 = f1_score(y_test, pred)

    print(test_cm)
    print('정확도', round(test_acc*100,2))
    print('정밀도', round(test_prc*100,2))
    print('재현율', round(test_rcll*100,2))
    print('F1',  round(test_f1*100,2))

#### GBM

#### GradientBoosting  X_train_over로


In [26]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score


gb_clf_by_over = GradientBoostingClassifier(random_state=11)
gb_clf_by_over.fit(X_train_over, y_train_over)


gb_clf_by_over_cross = cross_val_score(gb_clf_by_over, X_train_over, y_train_over, scoring='recall', cv=5)

print(gb_clf_by_over_cross)
print()
print(gb_clf_by_over_cross.mean())

[0.53386273 0.95818294 0.95834336 0.96107056 0.96719339]

0.8757305954386247


In [27]:
pred = gb_clf_by_over.predict(X_test)

get_clf_eval(y_test,pred)

[[23810 22860]
 [ 1666  2603]]
정확도 51.85
정밀도 10.22
재현율 60.97
F1 17.51


In [28]:
import joblib

joblib.dump(gb_clf_by_over, './gbm_over_under_sampling.pkl')

['./gbm_over_under_sampling.pkl']

#### GradientBoosting  X_train_smote로



In [29]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(random_state=11)
gb_clf.fit(X_train_smote, y_train_smote)

GradientBoostingClassifier(random_state=11)

In [ ]:
from sklearn.model_selection import cross_val_score

gb_clf_cross = cross_val_score(gb_clf, X_train_smote, y_train_smote, scoring='recall', cv=5)

print(gb_clf_cross)
print()
print(gb_clf_cross.mean())

In [ ]:
pred = gb_clf.predict(X_test)

get_clf_eval(y_test,pred)

In [ ]:
import joblib

joblib.dump(gb_clf, './gbm_over_sampling.pkl')

#### GradientBoostingClassifier yes hyperparameter

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf_2 = GradientBoostingClassifier(n_estimators=200, learning_rate=0.1, min_samples_leaf=30, max_depth=5, random_state=11)

gb_clf.fit(X_train_smote, y_train_smote)

In [ ]:
from sklearn.model_selection import cross_val_score

gb_clf_cross = cross_val_score(gb_clf_2, X_train_smote, y_train_smote, scoring='recall', cv=5)

print(gb_clf_cross)
print()
print(gb_clf_cross.mean())

In [ ]:
pred = gb_clf2.predict(X_test)

get_clf_eval(y_test,pred)

In [ ]:
import joblib

joblib.dump(gb_clf, './gbm_hyper.pkl')

#### LightGBMClassifier

In [12]:
from lightgbm import LGBMClassifier

l_gbm = LGBMClassifier(n_estimators=200, learning_rate=0.1, max_depth=3,random_state=11)

evals = [(X_test, y_test)]
l_gbm.fit(X_train_smote, y_train_smote, 
        early_stopping_rounds=300, 
        eval_metric="logloss", eval_set=evals, verbose=True)

/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.10/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's binary_logloss: 0.685265
[2]	valid_0's binary_logloss: 0.676877
[3]	valid_0's binary_logloss: 0.669282
[4]	valid_0's binary_logloss: 0.662689
[5]	valid_0's binary_logloss: 0.670581
[6]	valid_0's binary_logloss: 0.687902
[7]	valid_0's binary_logloss: 0.734632
[8]	valid_0's binary_logloss: 0.728033
[9]	valid_0's binary_logloss: 0.722065
[10]	valid_0's binary_logloss: 0.713946
[11]	valid_0's binary_logloss: 0.707682
[12]	valid_0's binary_logloss: 0.703501
[13]	valid_0's binary_logloss: 0.724799
[14]	valid_0's binary_logloss: 0.76888
[15]	valid_0's binary_logloss: 0.762212
[16]	valid_0's binary_logloss: 0.755745
[17]	valid_0's binary_logloss: 0.746055
[18]	valid_0's binary_logloss: 0.741401
[19]	valid_0's binary_logloss: 0.735453
[20]	valid_0's binary_logloss: 0.729685
[21]	valid_0's binary_logloss: 0.724666
[22]	valid_0's binary_logloss: 0.719391
[23]	valid_0's binary_logloss: 0.71298
[24]	valid_0's binary_logloss: 0.707546
[25]	valid_0's binary_logloss: 0.703025
[26]	valid_

LGBMClassifier(max_depth=3, n_estimators=200, random_state=11)

In [13]:
from sklearn.model_selection import cross_val_score

l_gbm_cross = cross_val_score(l_gbm, X_train_smote, y_train_smote, scoring='recall', cv=5)

print(l_gbm_cross)
print()
print(l_gbm_cross.mean())

[0.5425256  0.98144435 0.9813374  0.98195235 0.98077592]

0.8936071228042032


In [14]:
pred = l_gbm.predict(X_test)

get_clf_eval(y_test,pred)

[[31635 15035]
 [ 2365  1904]]
정확도 65.84
정밀도 11.24
재현율 44.6
F1 17.96


In [15]:
import joblib

joblib.dump(l_gbm, './light_gbm.pkl')

['./light_gbm.pkl']

#### XGBoost

In [16]:
from xgboost import XGBClassifier

xgb=XGBClassifier(booster='gbtree', n_estimators=200, learning_rate=0.1, max_depth=3, random_state=11, n_jobs=-1)

xgb.fit(X_train_smote, y_train_smote, early_stopping_rounds=300, eval_metric='logloss', eval_set = [(X_test, y_test)])

xgb.feature_importances_

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.68527
[1]	validation_0-logloss:0.67687
[2]	validation_0-logloss:0.66927
[3]	validation_0-logloss:0.66267
[4]	validation_0-logloss:0.67056
[5]	validation_0-logloss:0.68784
[6]	validation_0-logloss:0.73454
[7]	validation_0-logloss:0.72794
[8]	validation_0-logloss:0.72198
[9]	validation_0-logloss:0.71395
[10]	validation_0-logloss:0.70789
[11]	validation_0-logloss:0.70370
[12]	validation_0-logloss:0.72499
[13]	validation_0-logloss:0.76954
[14]	validation_0-logloss:0.75980
[15]	validation_0-logloss:0.75350
[16]	validation_0-logloss:0.74697
[17]	validation_0-logloss:0.74118
[18]	validation_0-logloss:0.73660
[19]	validation_0-logloss:0.73069
[20]	validation_0-logloss:0.72574
[21]	validation_0-logloss:0.71921
[22]	validation_0-logloss:0.74062
[23]	validation_0-logloss:0.73510
[24]	validation_0-logloss:0.72963
[25]	validation_0-logloss:0.72525
[26]	validation_0-logloss:0.72002
[27]	validation_0-logloss:0.73923
[28]	validation_0-logloss:0.78405
[29]	validation_0-loglos

array([8.8378444e-04, 4.2660609e-03, 4.8561827e-03, 1.0183393e-03,
       3.3351194e-02, 3.1192401e-02, 2.5266614e-02, 0.0000000e+00,
       0.0000000e+00, 4.6751335e-02, 9.0936344e-05, 2.0012047e-02,
       7.3461798e-03, 4.2163748e-03, 3.6641075e-03, 7.6899440e-03,
       2.1371536e-03, 2.5731728e-03, 1.4513255e-02, 0.0000000e+00,
       2.0919884e-02, 6.8115350e-03, 3.9040013e-03, 3.0900105e-03,
       9.0463506e-03, 2.0914353e-02, 2.8373161e-02, 2.1610243e-02,
       1.8112835e-02, 1.5776348e-03, 6.8846382e-03, 0.0000000e+00,
       2.0358456e-02, 2.6736779e-03, 2.4329947e-02, 1.5658060e-02,
       5.5553271e-03, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       2.7937663e-03, 5.9951916e-03, 2.1055505e-02, 3.5681434e-02,
       4.1793799e-03, 4.9765971e-03, 2.7384913e-02, 1.3467627e-02,
       4.4842124e-02, 3.5061821e-02, 6.7200200e-03, 6.2325906e-02,
       0.0000000e+00, 2.6702858e-03, 7.4820551e-03, 2.6024854e-02,
       1.6355211e-02, 1.4867206e-02, 1.0999368e-02, 1.7111279e

In [17]:
from sklearn.model_selection import cross_val_score

xg_cross = cross_val_score(xgb, X_train_smote, y_train_smote, scoring='recall', cv=5)

print(xg_cross)
print()
print(xg_cross.mean())

[0.54145611 0.98074918 0.98216625 0.98203257 0.98107002]

0.8934948263415416


In [18]:
pred = xgb.predict(X_test)

get_clf_eval(y_test, pred)

[[31635 15035]
 [ 2365  1904]]
정확도 65.84
정밀도 11.24
재현율 44.6
F1 17.96


In [20]:
import joblib

joblib.dump(xg_cross, './xgboost.pkl')

['./xgboost.pkl']

#### XGBoost feature_importances_ 확인하고 중요도 높은 피처 15개로 다시 학습 & 검증

In [ ]:
# pd.options.display.float_format = '{:.5f}'.format

In [ ]:
# df_feature_importance_xg = pd.DataFrame()
# df_feature_importance_xg["columns"] = X_train_smote.columns
# df_feature_importance_xg["importance"] = xgb.feature_importances_

# df_feature_importance_xg.sort_values("importance", ascending=False, inplace=True)
# df_feature_importance_xg.head(15)["columns"].values

In [ ]:
# xg_feature_importance_15 = X_train_smote[['NAME_INCOME_TYPE_2', 'NAME_TYPE_SUITE_2',
#        'NAME_CONTRACT_TYPE_Cash loans', 'NAME_TYPE_SUITE_1',
#        'ORGANIZATION_TYPE_3', 'NAME_TYPE_SUITE_3',
#        'NAME_CONTRACT_TYPE_Revolving loans', 'FLAG_OWN_REALTY_Y',
#        'DAYS_ID_PUBLISH_int', 'HOUR_APPR_PROCESS_START_2',
#        'DAYS_LAST_PHONE_CHANGE_int', 'WEEKDAY_APPR_PROCESS_START_TUESDAY',
#        'NAME_EDUCATION_TYPE', 'WEEKDAY_APPR_PROCESS_START_MONDAY',
#        'WEEKDAY_APPR_PROCESS_START_FRIDAY']]

In [ ]:
# X_test_feature_importance = X_test[['NAME_INCOME_TYPE_2', 'NAME_TYPE_SUITE_2',
#        'NAME_CONTRACT_TYPE_Cash loans', 'NAME_TYPE_SUITE_1',
#        'ORGANIZATION_TYPE_3', 'NAME_TYPE_SUITE_3',
#        'NAME_CONTRACT_TYPE_Revolving loans', 'FLAG_OWN_REALTY_Y',
#        'DAYS_ID_PUBLISH_int', 'HOUR_APPR_PROCESS_START_2',
#        'DAYS_LAST_PHONE_CHANGE_int', 'WEEKDAY_APPR_PROCESS_START_TUESDAY',
#        'NAME_EDUCATION_TYPE', 'WEEKDAY_APPR_PROCESS_START_MONDAY',
#        'WEEKDAY_APPR_PROCESS_START_FRIDAY']]

In [ ]:
# from xgboost import XGBClassifier

# xgb=XGBClassifier(booster='gbtree', n_estimators=200, learning_rate=0.1, max_depth=3, random_state=11, n_jobs=-1)

# xgb.fit(xg_feature_importance_15, y_train_smote, early_stopping_rounds=300, eval_metric='logloss', eval_set = [(X_test_feature_importance, y_test)])

In [ ]:
# from sklearn.model_selection import cross_val_score

# xg_cross = cross_val_score(xgb, xg_feature_importance_15, y_train_smote, scoring='recall', cv=5)

# print(xg_cross)
# print()
# print(xg_cross.mean())

In [ ]:
# pred = xgb.predict(X_test_feature_importance)

# get_clf_eval(y_test, pred)

#### AdaBoost

##### Ada no hyper parameter 이미 저장

In [ ]:
# from sklearn.ensemble import AdaBoostClassifier

# ada = AdaBoostClassifier(n_estimators=200, learning_rate=0.1, random_state=11)

# ada.fit(X_train_smote, y_train_smote)

In [ ]:
# from sklearn.model_selection import cross_val_score

# ada_cross = cross_val_score(ada, X_train_smote, y_train_smote, scoring='recall', cv=5)

# print(ada_cross)
# print()
# print(ada_cross.mean())

In [ ]:
# pred = ada.predict(X_test)

# get_clf_eval(y_test, pred)

In [ ]:
# import joblib

# joblib.dump(gb_clf, './xgboost.pkl')

##### Ada hyper parameter

In [21]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(max_depth=3, min_samples_leaf=5)

ada_max_depth3 = AdaBoostClassifier(estimator=dt_clf, n_estimators=200, learning_rate=0.1, random_state=11)

ada_max_depth3.fit(X_train_smote, y_train_smote)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3,
                                                    min_samples_leaf=5),
                   learning_rate=0.1, n_estimators=200, random_state=11)

In [ ]:
from sklearn.model_selection import cross_val_score

ada_cross = cross_val_score(ada_max_depth3, X_train_smote, y_train_smote, scoring='recall', cv=5)

print(ada_cross)
print()
print(ada_cross.mean())

In [ ]:
pred = ada_max_depth3.predict(X_test)

get_clf_eval(y_test, pred)

In [ ]:
import joblib

joblib.dump(ada_max_depth3, './ada_hyper_param.pkl')